In [ ]:
import requests

url = 'http://127.0.0.1:5000/v1/test'
headers = {'user_id': '111'}
response = requests.get(url, headers=headers)

print(response.json())

In [7]:
from src.config.fetch_app_config import fetch_app_config


app_config = fetch_app_config("10101010")

app_config.config_values["ai_search_config"]["aisearch_service_endpoint"]
        
#isearch_service_endpoint = app_config.config_values.ai_search_config.

'CMS_AZURE_AI_SEARCH_ENDPOINT'